<a href="https://colab.research.google.com/github/hank199599/data_science_from_scratch_reading_log/blob/main/Chapter12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 最近鄰分類法(Nearest neighbors)
* 要有某種距離的概念
* 有個假設：距離較近的點彼此間會較類似

In [3]:
# 計算票數的函式
from typing import List
from collections import Counter

def raw_majority_vote(labels:List[str])->str:
  votes = Counter(labels)
  winner,_ = votes.most_common(1)[0]
  return winner

In [ ]:
def major_votes(labels:List[str])->str:
  """假設labels中的資料已事先依據從進到遠的順序排序"""
    vote_conuts = Counter(labels)
    winner,winner_count = vote_conuts.most_common(1)[0]
    num_winners = len([count for countin vote_conuts.values() if count == winner_count])

    if num_winners == 1:
      return winner
    else:
      return major_votes(labels[:-1])

In [23]:

from typing import List

Vector = List[float]

def subtrate( v:Vector, w:Vector) -> Vector:
  assert len(v) == len(w) #兩個向量必須有相同的維度

  return [ v_i-w_i for v_i,w_i in zip(v,w)]

def magnitude(v:Vector)->float:
  return math.aqrt(sum_of_squares(v)) #math.sqrt 是計算平方根的一個函式

def distance(v:Vector,w:Vector) -> float:
  return magnitude(subtract(v,w))

In [24]:
from typing import NamedTuple

class LabeledPoint(NamedTuple):
  point:Vector
  label:str

def knn_classify(k:int,
         label_points:List[LabeledPoint],
         new_point:Vector) ->str:
  
  #針對這些已標示分類標籤的點，根據距離進到遠的順序進行排序
  by_distance = sorted(label_points,
             key=lambda lp:dustance(lp.point,new_point))
  
  #找出k個最近點所對應的分類標籤
  k_nearest_labels = [lp.label for lp in by_distance[k]]

  #讓他們進行投票
  return majority_vote(k_nearest_labels)

## 範例：鳶尾花資料集 
在機器學習方面常被運用的資料集。  
這個資料集包含150朵鳶尾花的測量值，每一朵花都有對應的萼片長度與寬度、以及所屬品種。


下載資料集

In [19]:
from dateutil.parser import parse
import csv
data = requests.get("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data").content

with open('iris.data','w') as f:
  reader = csv.reader(f)
  reader

相應資料欄位如下：  
sepal_length, sepal_width, petal length, petal_width, class   
(花瓣長度,花辦寬度,等片長度,對片寬度,品種)

In [27]:
from typing import Dict
import csv
from collections import defaultdict

def parse_iris_row(row:List[str])->LabeledPoint:
  """
  sepal_length, sepal_width, petal length, petal_width, class
  (花瓣長度,花辦寬度,等片長度,對片寬度,品種)
  """
  measurements=[float(value) for value in row[:-1]]
  #像"Iris-virginica"這樣的品種名稱，我們只取其中的"virginica"
  label = row[-1].split("-")[1]

  return LabeledPoint(measurements,label)

with open('iris.data') as f:
  reader = csv.reader(f)
  iris_data = [parse_iris_row(row) for row in reader]

#我們只處理標有品種名稱的資料點，這樣才能順利畫出圖形
points_by_species:Dict[str,List[Vector]] = defaultdict(list)
for iris in iris_data:
  points_by_species[iris.label].append(iris.point)